In [192]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.model_selection import cross_validate, train_test_split
from statistics import mean 
import pandas as pd
import numpy as np

In [193]:
d = pd.read_csv('processedData.csv')

In [194]:
y = d['class'] #sets y to be class column 
X = d.iloc[:,0:(d.shape[1]-1)] #sets X to be dataset with class column removed 

In [195]:
#double checking that X does not have the label column (leaving the labels as a feature is a common mistake): 
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
1,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,20,15,16,12,5,-1,-18,-37,-47,-48
11496,-47,-11,28,77,141,211,246,240,193,136,...,-94,-65,-33,-7,14,27,48,77,117,170
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,114,121,135,148,143,116,86,68,59,55


-------------------

In [196]:
#WARNING: MAKING X AND Y SMALLER FOR NOW 
#We'll want to use the full dataset when reporting final numbers
#I'd be nice if we could do 5 fold CV, but that will take a very very long time if we're using the full dataset

In [197]:
X = X.iloc[0:2000,:]

In [198]:
y = y.iloc[0:2000]

In [199]:
print(X.shape)
print(y.shape)

(2000, 178)
(2000,)


------------

In [208]:
#Train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [209]:
##SVM 
from sklearn import svm
modelSVM = svm.SVC(gamma=0.001, C=100.) 
modelSVM.fit(X_train, y_train)
print(classification_report(y_test,modelSVM.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90       324
           1       0.00      0.00      0.00        76

    accuracy                           0.81       400
   macro avg       0.41      0.50      0.45       400
weighted avg       0.66      0.81      0.72       400



c:\program files\python37\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [202]:
#80% might seem  pretty bad considering the fact that we only have two classes 
#and one of them makes up 80% of the dataset...
#However, the precision and recall and f-score aren't 0, which means 
#the model is still learning at least.
#Still, everything is quite low. 

In [203]:
#I proceed to try some more simple non-deep learning models: 

In [204]:
##Logistic regression: 
from sklearn import linear_model
modelLR = linear_model.LogisticRegression(C=1e5, max_iter=1000)    
modelLR.fit(X_train, y_train)
print(classification_report(y_test, modelLR.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       318
           1       0.76      0.23      0.36        82

    accuracy                           0.83       400
   macro avg       0.80      0.61      0.63       400
weighted avg       0.82      0.83      0.79       400



c:\program files\python37\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [205]:
##Random forests
from sklearn.ensemble import RandomForestClassifier
modelRF=RandomForestClassifier(n_estimators=1000)   
modelRF.fit(X_train, y_train)
print(classification_report(y_test, modelRF.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       318
           1       0.90      0.95      0.92        82

    accuracy                           0.97       400
   macro avg       0.94      0.96      0.95       400
weighted avg       0.97      0.97      0.97       400



In [210]:
##Neural network: 
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

modelNeuralNet = MLPClassifier(hidden_layer_sizes=(5)) 
modelNeuralNet.fit(X_train, y_train)
print(classification_report(y_test,modelNeuralNet.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       324
           1       0.91      0.64      0.75        76

    accuracy                           0.92       400
   macro avg       0.91      0.81      0.85       400
weighted avg       0.92      0.92      0.91       400



c:\program files\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [207]:
#Definitely room for improvement on all 4 baselines.
#The slower ones (RF, NN) have higher performance as expected
#I tried out some other models. kNN performs similarly to SVM and logistic regression, 
#and adaboost and GBMs perform similarly to the random forest. 
#But to keep our presentation from getting cluttered, let's just stick to these
#4 baseline models 

In [164]:
#Cross validations (could only do 3 fold):

In [165]:
SVM_cv_results = cross_validate(modelSVM, X, y, cv=3) 

In [165]:
LR_cv_results = cross_validate(modelLR, X, y, cv=3)

In [165]:
NeuralNet_cv_results = cross_validate(modelNeuralNet, X, y, cv=3)

In [165]:
RF_cv_results = cross_validate(modelRF, X, y, cv=3)

c:\program files\python37\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\program files\python37\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic

In [213]:
print("Accuracies:")
print("SVM_cv_results: " + str(mean(SVM_cv_results['test_score'])))
print("LR_cv_results: " + str(mean(LR_cv_results['test_score'])))
print("NeuralNet_cv_results: " + str(mean(NeuralNet_cv_results['test_score'])))
print("RandomForest_cv_results: " + str(mean(RF_cv_results['test_score'])))

Accuracies:
SVM_cv_results: 0.8040004022013018
LR_cv_results: 0.8210031620826224
NeuralNet_cv_results: 0.740490115302709
RandomForest_cv_results: 0.9605002303652979


In [214]:
print("Training times:")

Training times:


In [211]:
SVM_cv_results

{'fit_time': array([0.5652864 , 0.59992695, 0.51484203]),
 'score_time': array([0.25506043, 0.21972632, 0.25529981]),
 'test_score': array([0.8035982, 0.8035982, 0.8048048])}